In [ ]:
import numpy as np

def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation 
    of the previous layer.
    
    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)
    
    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """

    s = np.multiply(a_slice_prev, W)
    Z = np.sum(s)
    Z = Z + float(b)

    return Z

def conv_forward_example(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, 
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """

    # Take parameters and initialize
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    f, f, n_C_prev, n_C = W.shape
    stride = hparameters["stride"]
    pad = hparameters['pad']
    n_H = int((n_H_prev+2*pad-f)/stride + 1)
    n_W = int((n_W_prev+2*pad-f)/stride + 1)
    Z = np.zeros((m, n_H, n_W, n_C))

    A_prev_pad = np.pad(A_prev, ((0,0), (pad,pad), (pad,pad), (0,0)), mode='constant', constant_values = (0,0))
    
    for i in range(m):               
        a_prev_pad = A_prev_pad[i,:,:,:]                
        for h in range(n_H):           
            vert_start = h*stride
            vert_end = h*stride+f
            for w in range(n_W):       
                horiz_start = w*stride
                horiz_end = w*stride+f
                for c in range(n_C):   

                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end,:]
                    weights = W[:,:,:,c]
                    biases = b[:,:,:,c]

                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)
            
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

def pool_forward(A_prev, hparameters, mode = "max"):

    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))              

    for i in range(m):                         # loop over the training examples
        for h in range(0,n_H, stride):         # loop on the vertical axis of the output volume
            vert_start = h
            vert_end = h+f
            
            for w in range(0,n_W, stride):                 # loop on the horizontal axis of the output volume
                horiz_start = w
                horiz_end = w+f
                
                for c in range (n_C):            # loop over the channels of the output volume
                    
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]  
                    
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    

    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    return A, cache

def conv_forward_multiple_examples(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function through m examples
    
    Arguments:
    A_prev -- output activations of the previous layer, 
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (m, f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (m, 1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function (A_prev, W, b, hparameters)
    """
    # initialize
    m = A_prev.shape[0]
    Z = [None] * m
    cache = [None]* m 
    for i in range(m):
        W_example = W[i,:,:,:,:]
        b_example = b[i,:,:,:,:]
        Z[i], _ = conv_forward_example(A_prev, W_example, b_example, hparameters)

    A = np.array(Z[i])
    print(A.shape)
    cache_update = (A_prev, W, b, hparameters)
    return A, cache_update